# Libs

In [ ]:
import os
# while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

In [3]:
import os
import regex as re
import random
import json
import pandas as pd
import numpy as np

from itertools import combinations
from itertools import product  

In [4]:
base_model_id = "FacebookAI/xlm-roberta-base"

# Load tokenizer

In [5]:
import torch
from transformers import AutoTokenizer, AutoModel

ModuleNotFoundError: No module named 'transformers'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Chunking

In [7]:
# data_path = "/kaggle/input/vn-law-test/data"
data_path = "data/"

MAX_LEN = 512
OVERLAP = 50

In [8]:
files = os.listdir(data_path)
print(files)
data_stored = []

['BaoHiem.json', 'BoMayHanhChinh.json', 'CoCauToChuc.json', 'CongNgheThongTin.json', 'CongNghiep.json', 'DanSu.json', 'DichVuPhapLy.json', 'DoanhNghiep.json', 'GiaoDuc.json', 'GiaoThongVanTai.json', 'LaoDong.json', 'NhaDat.json', 'TaiChinh', 'TaiChinhNganHang.json']


In [9]:
# def preprocess_phrase_chunking(paragraphs):
#     for i in range(len(paragraphs)):
#         s = paragraphs[i]
#         s = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '', s)
#         s = re.sub('Độc lập - Tự do - Hạnh phúc', '', s)
#         s = re.sub('[- ]+', ' ', s)
#         s = s.strip()
#         paragraphs[i] = s

#     for i in range(3):
#         for i in range(len(paragraphs)):
#             if len(paragraphs[i]) < 1:
#                 paragraphs.pop(i)
#                 break
    
#     return paragraphs

# def count_words(input_text):
#     return len(input_text.split())

# def markdown_chunking(paragraphs, max_len=512, overlap=100):
#     lst_chunks = []
#     current_chunk = ''
#     for p in paragraphs:
#         if count_words(current_chunk) + count_words(p) <= max_len:
#             if len(current_chunk) == 0:
#                 current_chunk = p
#             else:
#                 current_chunk = "\n".join([current_chunk, p])
#         else:
#             lst_chunks.append(current_chunk)
#             current_chunk = " ".join(current_chunk.split(" ")[-100:]) + "\n" + p

#     lst_chunks.append(current_chunk)
#     return lst_chunks 

# def tokenizer_chunking(content, tokenizer):
#     pass


### test 1 chunk

In [10]:
file = files[0]

with open(f"{data_path}/{file}", "r", encoding="utf-8") as f:
    data = json.load(f)

data = data[0]

In [11]:
def preprocess_phrase_chunking(content: str):
    content = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '', content)
    content = re.sub('Độc lập - Tự do - Hạnh phúc', '', content)
    content = re.sub('[- ]+', ' ', content)
    content = content.strip()
    return content

content = data['content']
content = "\n".join(content)
content = preprocess_phrase_chunking(content)

In [12]:
from langchain_text_splitters import CharacterTextSplitter

In [13]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=512,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [14]:
texts = text_splitter.split_text(content)

Created a chunk of size 570, which is longer than the specified 512
Created a chunk of size 518, which is longer than the specified 512


In [40]:
print(texts[0])

HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA 
 
Số: 06/2024/NQ HĐND
Khánh Hòa, ngày 12 tháng 7 năm 2024
NGHỊ QUYẾT
VỀ VIỆC BAN HÀNH MỨC GIÁ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH KHÔNG THUỘC PHẠM VI THANH TOÁN CỦA QUỸ BẢO HIỂM Y TẾ MÀ KHÔNG PHẢI LÀ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH THEO YÊU CẦU TẠI CÁC CƠ SỞ KHÁM BỆNH, CHỮA BỆNH CỦA NHÀ NƯỚC TRÊN ĐỊA BÀN TỈNH KHÁNH HÒA
HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA KHÓA VII, KỲ HỌP THỨ 14
Căn cứ Luật Tổ chức chính quyền địa phương ngày 19 tháng 6 năm 2015;


In [ ]:
tokens = tokenizer.encode_plus(
    texts[7], 
    truncation=True, 
    padding='max_length',
    return_tensors='pt'
    )
tokens['input_ids'].shape


torch.Size([1, 512])

torch.Size([1, 512])

In [ ]:
# def tokenizer_chunking(content, tokenizer, max_len=512, overlap=50):
#     tokens = tokenizer.encode(content, add_special_tokens=False)
#     chunks = []
#     start = 0

#     while start < len(tokens):
#         end = min(start + max_len, len(tokens))
#         chunk = tokens[start:end]
#         chunks.append(chunk)
#         start += max_len - overlap

#     return chunks

# # Example usage
# tokenized_chunks = tokenizer_chunking(content, tokenizer, max_len=MAX_LEN, overlap=OVERLAP)
# print(f"Number of chunks: {len(tokenized_chunks)}")

512